In [ ]:
%reload_ext dotenv
%dotenv .env

In [ ]:
import os
import sqlite3

from classes import TramStop, TramPassage, DayType

connection = sqlite3.connect(os.environ.get('DATABASE_NAME'))
cursor = connection.cursor()

In [ ]:
cursor.execute("SELECT id, name, latitude, longitude FROM tram_stops")

tram_stops: dict[int, TramStop] = {
    item[0]: TramStop(*item) for item in cursor.fetchall()
}

tram_stops

# Line 1

In [ ]:
day, line_id, line_variant = DayType.WEEKDAY, 1, 1

cursor.execute("""
    SELECT
        tram_stops.id,
        tram_departures.id,
        tram_departures.stop_index,
        tram_departures.hour,
        tram_departures.minute
    FROM tram_departures
        JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
        JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
    WHERE tram_departures.day = ? AND tram_line_variants.id = ?
    ORDER BY
        tram_departures.stop_index DESC,
        tram_departures.hour DESC,
        tram_departures.minute DESC
""", (day.value, line_variant))

departures = cursor.fetchall()
departure_stop = {item[1]: tram_stops[item[0]] for item in departures}

departure_stop

In [ ]:
passages: list[TramPassage] = []
departure_passage: dict[int, TramPassage] = {}

for stop in filter(lambda x: x[1] not in departure_passage, departures):
    passage = TramPassage(line_id, day)
    passages.append(passage)

    for stop_id, departure_id, stop_index, hour, minute in filter(
        lambda x: x == stop or x[2] == stop_index - 1 and (x[3] < hour or x[3] == hour and x[4] < minute),
        departures
    ):
        if departure_id in departure_passage:
            earlier_passage = departure_passage[departure_id]
            for _ in range(earlier_passage.departures.index(departure_id), len(earlier_passage.departures)):
                del departure_passage[earlier_passage.departures.pop()]

        passage.add_stop(departure_id, departure_stop[departure_id], hour, minute)
        departure_passage[departure_id] = passage

[item.reverse_stop_sequence for item in passages]

## Line 50

#### Borek Fałęcki → Górka Narodowa P+R

In [ ]:
day, line_id, line_variant = DayType.WEEKDAY, 2, 4

cursor.execute("""
    SELECT
        tram_stops.id,
        tram_departures.id,
        tram_departures.stop_index,
        tram_departures.hour,
        tram_departures.minute
    FROM tram_departures
        JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
        JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
    WHERE tram_departures.day = ? AND tram_line_variants.id = ?
    ORDER BY
        tram_departures.stop_index DESC,
        tram_departures.hour DESC,
        tram_departures.minute DESC
""", (day.value, line_variant))

departures = cursor.fetchall()
departure_stop = {item[1]: tram_stops[item[0]] for item in departures}
departure_time = {item[1]: (item[3], item[4]) for item in departures}

In [ ]:
passages: list[TramPassage] = []
departure_passage: dict[int, TramPassage] = {}

highest_stop_index = max(map(lambda x: x[2], departures))
for initial_stop in filter(lambda x: x[1] not in departure_passage, departures):
    passage = TramPassage(line_id, day)
    passages.append(passage)

    if initial_stop[2] < highest_stop_index:
        print("Second-last stop somewhere else:", tram_stops[initial_stop[0]].name)

    for stop_id, departure_id, stop_index, hour, minute in filter(
        lambda x: x is initial_stop or x[2] == stop_index - 1 and (x[3] < hour or x[3] == hour and x[4] < minute),
        departures
    ):
        if departure_id in departure_passage:
            earlier_passage = departure_passage[departure_id]
            for _ in range(earlier_passage.departures.index(departure_id), len(earlier_passage.departures)):
                del departure_passage[earlier_passage.departures.pop()]

        passage.add_stop(departure_id, departure_stop[departure_id], hour, minute)
        departure_passage[departure_id] = passage

[item.reverse_stop_sequence for item in passages]

#### Górka Narodowa P+R → Borek Fałęcki

In [ ]:
day, line_id, line_variant = DayType.WEEKDAY, 2, 3

cursor.execute("""
    SELECT
        tram_stops.id,
        tram_departures.id,
        tram_departures.stop_index,
        tram_departures.hour,
        tram_departures.minute
    FROM tram_departures
        JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
        JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
    WHERE tram_departures.day = ? AND tram_line_variants.id = ?
    ORDER BY
        tram_departures.stop_index DESC,
        tram_departures.hour DESC,
        tram_departures.minute DESC
""", (day.value, line_variant))

departures = cursor.fetchall()
departure_stop = {item[1]: tram_stops[item[0]] for item in departures}
departure_time = {item[1]: (item[3], item[4]) for item in departures}

In [ ]:
passages: list[TramPassage] = []
departure_passage: dict[int, TramPassage] = {}

highest_stop_index = max(map(lambda x: x[2], departures))
for initial_stop in filter(lambda x: x[1] not in departure_passage, departures):
    passage = TramPassage(line_id, day)
    passages.append(passage)

    # Add departure for early end of passage
    if initial_stop[2] < highest_stop_index:
        print("Second last stop somewhere else:", tram_stops[initial_stop[0]])
        cursor.execute("""
            SELECT DISTINCT tram_stops.id
            FROM tram_departures
                JOIN tram_line_variants ON tram_departures.tram_line_variant_id = tram_line_variants.id
                JOIN tram_stops ON tram_departures.tram_stop_id = tram_stops.id
            WHERE tram_departures.day = 'P' AND tram_line_variants.id = 3 AND tram_departures.stop_index = ?
        """, (initial_stop[2] + 1,))

        stop_id = cursor.fetchone()[0]
        hour, minute = (initial_stop[3] + (initial_stop[4] + 1) // 60) % 24, (initial_stop[4] + 1) % 60

        cursor.execute(
            """
                INSERT INTO tram_departures (tram_stop_id, tram_line_variant_id, stop_index, day, hour, minute)
                VALUES (?, ?, ?, ?, ?, ?)
                RETURNING id
            """,
            (
                stop_id,
                line_variant,
                initial_stop[2] + 1,
                day.value,
                hour,
                minute
            )
        )

        departure_id = cursor.fetchone()[0]
        departure_stop[departure_id] = tram_stops[stop_id]
        departure_time[departure_id] = (hour, minute)

        passage.add_stop(departure_id, departure_stop[departure_id], hour, minute)

    for stop_id, departure_id, stop_index, hour, minute in filter(
        lambda x: x is initial_stop or x[2] == stop_index - 1 and (x[3] < hour or x[3] == hour and x[4] < minute),
        departures
    ):
        if departure_id in departure_passage:
            earlier_passage = departure_passage[departure_id]
            for _ in range(earlier_passage.get_departure_index(departure_id), len(earlier_passage)):
                del departure_passage[earlier_passage.departures.pop()]

        passage.add_stop(departure_id, departure_stop[departure_id], hour, minute)
        departure_passage[departure_id] = passage

    connection.commit()

[item.reverse_stop_sequence for item in passages]